In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [14]:
use_db = pd.read_csv("output/db_smiles_preproc.csv")

### Simple classification with Smile Features
Considering a threshold of 1mg/L of mortality

In [15]:
X = use_db.copy()
X = X[[
 'exposure_type',
 'obs_duration_mean',
 'conc1_type',
 'species',
 'atom_number',
 'bonds_number',
  'ring_numer',
  'Mol', 
  'MorganDensity', 'LogP']]

In [16]:
X['exposure_type'].unique()

array(['S', 'F', 'R', 'AQUA', 'P', 'IP', 'E', 'SP', 'O', 'FD'],
      dtype=object)

In [17]:
X['conc1_type'].unique()

array(['T', 'D', 'L', 'F', 'A', 'U'], dtype=object)

In [18]:
len(X['species'].unique())

489

For this  vaiables we decided to transform than by an OrdinalEncoder, in order to have all numnerical variables

In [19]:
enc = OrdinalEncoder(dtype=int)
enc.fit(X[["exposure_type", "conc1_type","species"]])
X[["exposure_type", "conc1_type","species"]] = enc.transform(X[["exposure_type", "conc1_type","species"]]) + 1

In [20]:
X.head()

,exposure_type,obs_duration_mean,conc1_type,species,atom_number,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,9,0.028036,5,40,0.167949,0.0,0.0,0.10092,0.666667,-5.9945
1,9,-0.730874,5,208,0.167949,0.0,0.0,0.10092,0.666667,-5.9945
2,3,0.798163,5,313,0.167949,0.0,0.0,0.10092,0.666667,-5.9945
3,9,0.798163,5,475,0.167949,0.0,0.0,0.10092,0.666667,-5.9945
4,3,3.018921,5,313,0.167949,0.0,0.0,0.10092,0.666667,-5.9945


We also devide the label, in order to have a binary classification

In [21]:
y = use_db[["conc1_mean"]].copy().values
y = np.where(y > 1, 1, 0)

In [22]:
print(len(y[y==0]), len(y[y==1]))

22551 31101


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## KNN

We first implement a classification by a K Neighbors Classifier

In [24]:
neigh = KNeighborsClassifier()
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
accuracy_score(y_test, y_pred)

0.84 is the accuracy on tis first model

## Random Forest

In [30]:
forest = RandomForestClassifier(n_estimators = 100)
forest.fit(X_train, y_train.ravel())
y_pred = forest.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [62]:
accuracy_score(y_test, y_pred)

0.9128774928774929

This is the second model we have implemented, with an accurancy of the 0.91. which for the moment is not bad

## Gradient Boosting

In [64]:
grboost = GradientBoostingClassifier()
grboost.fit(X_train, y_train.ravel())
y_pred = grboost.predict(X_test)

In [65]:
accuracy_score(y_test, y_pred)

0.8338461538461538

In [ ]:
# grid = GridSearchCV(forest, param_grid={"n_estimators": list(range(1, 500))}, scoring="accuracy", cv=5)
# grid.fit(X_train, y_train.ravel())